<a href="https://colab.research.google.com/github/YugantGotmare/EmoAudio-Decode-Emotions-from-Spoken-Words/blob/main/Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/  modelForPrediction1.sav
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [ ]:
!pip install librosa==0.9.1

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)  # Normalize MFCCs
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
# Example of pitch shifting augmentation
def pitch_shift(audio, sample_rate, max_steps=2):
    steps = np.random.randint(-max_steps, max_steps)
    return librosa.effects.pitch_shift(audio, sample_rate, n_steps=steps)

In [ ]:
# Load and augment the data
def load_data(test_size=0.2, augment=True):
    x, y = [], []
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        if feature is not None:
            x.append(feature)
            y.append(emotion)
            if augment:
                augmented_data = [feature]  # Original sample
                # Apply augmentation techniques (e.g., pitch shift, time stretch) here if needed
                x.extend(augmented_data * 3)  # Augment each sample 3 times
                y.extend([emotion] * len(augmented_data) * 3)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train, x_test, y_train, y_test = load_data(test_size=0.25, augment=True)

<ipython-input-4-dff4e24169eb>:17: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
<ipython-input-4-dff4e24169eb>:17: FutureWarning: Pass y=[0.00033569 0.00015259 0.00021362 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
<ipython-input-4-dff4e24169eb>:17: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.0517578e-05
 -3.0517578e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
<ipython-input-4-dff4e24169eb>:17: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ...  0.0000

In [ ]:
x_train

array([[-6.21423149e+00,  6.75192058e-01,  1.93497360e-01, ...,
         1.41615499e-04,  5.15730972e-05,  2.76150768e-05],
       [-6.20951891e+00,  6.43897951e-01,  4.20107208e-02, ...,
         1.21914018e-05,  6.27334703e-06,  3.88537273e-06],
       [-6.22112179e+00,  5.98123610e-01,  1.22297093e-01, ...,
         1.04338724e-04,  4.46671038e-05,  2.33882092e-05],
       ...,
       [-6.21099997e+00,  6.35641098e-01, -9.42331553e-03, ...,
         8.47551448e-04,  2.99824373e-04,  1.64255267e-04],
       [-6.22969484e+00,  5.52256882e-01,  1.04611143e-01, ...,
         9.33385600e-05,  5.54744256e-05,  3.07961745e-05],
       [-6.22283459e+00,  6.16275072e-01,  1.65710777e-01, ...,
         3.85820004e-03,  1.85037055e-03,  1.05014653e-03]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(2304, 768)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
# Initialize the Multi-Layer Perceptron Classifier
model = MLPClassifier(alpha=0.001, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(400, 200),
                      learning_rate='adaptive', max_iter=1000, random_state=9)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.001, batch_size=256, hidden_layer_sizes=(400, 200),
              learning_rate='adaptive', max_iter=1000, random_state=9)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['fearful', 'fearful', 'disgust', 'fearful', 'fearful', 'disgust',
       'fearful', 'fearful', 'calm', 'fearful', 'fearful', 'fearful',
       'fearful', 'happy', 'calm', 'happy', 'disgust', 'disgust',
       'fearful', 'disgust', 'fearful', 'disgust', 'happy', 'disgust',
       'disgust', 'calm', 'fearful', 'fearful', 'disgust', 'happy',
       'calm', 'calm', 'calm', 'disgust', 'calm', 'fearful', 'fearful',
       'calm', 'disgust', 'calm', 'disgust', 'disgust', 'calm', 'calm',
       'calm', 'calm', 'calm', 'fearful', 'calm', 'disgust', 'disgust',
       'calm', 'disgust', 'fearful', 'happy', 'fearful', 'calm',
       'fearful', 'fearful', 'disgust', 'disgust', 'calm', 'disgust',
       'fearful', 'happy', 'fearful', 'disgust', 'happy', 'fearful',
       'happy', 'fearful', 'calm', 'disgust', 'disgust', 'happy',
       'fearful', 'happy', 'disgust', 'calm', 'calm', 'fearful', 'happy',
       'happy', 'fearful', 'disgust', 'fearful', 'calm', 'calm', 'happy',
       'disgust', 

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 96.74%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# Calculate F1 score for each class
f1_scores = f1_score(y_test, y_pred, labels=observed_emotions, average=None)
for emotion, f1_score_value in zip(observed_emotions, f1_scores):
    print(f"F1 Score for {emotion}: {f1_score_value:.2f}")

F1 Score for calm: 0.96
F1 Score for happy: 0.99
F1 Score for fearful: 0.97
F1 Score for disgust: 0.94


In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.96438356, 0.94264339, 0.9721519 , 0.992     ])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,fearful,fearful
1,fearful,fearful
2,disgust,disgust
3,fearful,fearful
4,fearful,fearful
5,disgust,disgust
6,fearful,fearful
7,fearful,fearful
8,calm,calm
9,fearful,fearful


In [ ]:
pwd

'/content'

In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1_best_one.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1_best_one.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/voice_1.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

In [ ]:
# # # Save the trained model to a file
# # import pickle
# # with open('modelForPrediction1.sav', 'wb') as f:
# #     pickle.dump(model, f)

# # # Load the saved model and make predictions
# # filename = 'modelForPrediction1.sav'
# # loaded_model = pickle.load(open(filename, 'rb'))

# # Example: Make prediction for a single audio file
# example_feature = extract_feature("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_01/03-01-08-02-02-02-01.wav",
#                                   mfcc=True, chroma=True, mel=True)
# example_feature = example_feature.reshape(1, -1)
# prediction = loaded_model.predict(example_feature)
# print(f"Predicted emotion: {prediction[0]}")

In [ ]:
import librosa
import soundfile
import numpy as np
import pickle

# Emotions in the RAVDESS dataset
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)  # Normalize MFCCs
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

# Function to preprocess audio for prediction
def preprocess_audio(audio_file):
    feature = extract_feature(audio_file, mfcc=True, chroma=True, mel=True)
    return feature.reshape(1, -1)

# Replace 'your_audio_file.wav' with the path to your recorded audio file
recorded_feature = preprocess_audio('/content/voice_1.wav')

# Load the saved model from the file
filename = '/content/modelForPrediction1_best_one.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# Predict the emotion for the recorded audio
predicted_emotion = loaded_model.predict(recorded_feature)[0]

# Print the predicted emotion
print(f"Predicted emotion for your voice: {predicted_emotion}")